In [1]:
import nltk
from fuzzywuzzy import process
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
nlp = spacy.load('en_core_web_sm')

In [2]:
#Load File
data = pd.read_json("intents.json")
data

,intent_model
0,"{'context': 'appointment', 'utterances': ['I n..."
1,"{'context': 'insurance', 'utterances': ['Need ..."
2,"{'context': 'lab', 'utterances': ['Need to und..."


In [3]:
#Normalize
intents = pd.json_normalize(data['intent_model'])
intents

,context,utterances
0,appointment,"[I need help with doctor appointment, I am loo..."
1,insurance,"[Need help with insurance claim, What are the ..."
2,lab,"[Need to undergo diagnosis, How much is the la..."


In [4]:
#Flattened the utterences
corpus_flattened =[]
corpus_trans =[]
for utterences in intents['utterances']:
    for utter in utterences:
        corpus_flattened.append(utter)
        corpus_trans.append(utter.lower())
corpus_flattened

['I need help with doctor appointment',
 'I am looking for appointment',
 'Is the doctor available today',
 'Need help with insurance claim',
 'What are the insurance tie ups available',
 'I need help with claim settlement',
 'Need to undergo diagnosis',
 'How much is the lab test',
 'When can I get my lab report',
 'I am looking for lab tests']

In [5]:
#Vectorizer
stopwords_list=stopwords.words('english')
tf = TfidfVectorizer(max_features=2500, stop_words=stopwords_list)
x=tf.fit_transform(corpus_trans)
count_dense = x.todense()
count_dense[0]

matrix([[0.544699  , 0.        , 0.        , 0.        , 0.544699  ,
         0.        , 0.47654726, 0.        , 0.        , 0.        ,
         0.        , 0.42368467, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]])

In [6]:
def get_match(query):
    context=None
    for id, match_string in zip(intents.index, intents['utterances']):
        if query in match_string:
            context = intents['context'][id]
    return context

In [7]:
get_match('I need help with doctor appointment')

'appointment'

In [8]:
def get_item_match(query):
    max_score = 0
    best_match=None
    context=None
    y = tf.transform([query.lower()])
    max_score = np.max(cosine_similarity(y, count_dense))
    if max_score > 0.5:
         best_match = corpus_flattened[np.argmax(cosine_similarity(y, count_dense))]
         context = get_match(corpus_flattened[np.argmax(cosine_similarity(y, count_dense))])
    return max_score, best_match, context

In [9]:
print(get_item_match('I am 30 years old'))
print(get_item_match('i want to CLAIM!'))
print(get_item_match('Could you assist me for lab test?'))

(0.0, None, None)
(0.5446990040204331, 'Need help with insurance claim', 'insurance')
(0.7799515658651095, 'How much is the lab test', 'lab')


In [10]:
diseases = ['fever', 'cardiac', 'inflammation', 'throat pain', 'heart burn', 'head ache', 'allergies', 'conjunctivitis', 'diarrhea', 'stomach ache/pain']
diseases

['fever',
 'cardiac',
 'inflammation',
 'throat pain',
 'heart burn',
 'head ache',
 'allergies',
 'conjunctivitis',
 'diarrhea',
 'stomach ache/pain']

In [11]:
def detected_disease(query):
    if process.extract(query.lower(), diseases)[0][1] >= 70:
        disease = process.extract(query, diseases)[0][0].title()
    else:
        disease = None
    return disease

In [12]:
def duration(query):
    doc = nlp(query.lower())
    global days
    for ent in doc.ents:
        if (ent.label_=='DATE') or (ent.label_=='TIME') or (ent.label_=='ORG'):
            days = ent.text

In [13]:
def appointment_date(query):
    global time
    global date
    doc = nlp(query)
    for ent in doc.ents:
        if ent.label_=='DATE' or ent.label_=='EVENT':
            date = ent.text
        elif ent.label_=='TIME':
            time = ent.text

In [14]:
medical_test = ['ct scan', 'blood sugar test', 'urine test', 'cholesterol test', 'ecg', 'eeg']
price_list = [5018.00, 360.00, 600.00, 900.00, 3600.00, 1605.00]

In [15]:
def labtest_price(query):
    global count
    tests_append=[]
    if "all" in query.lower().split(',') or "all" in query.lower().split(', '):
        count = 12433.0
        tests_append.append("All Tests")
        return tests_append
    else:
        for i, x in enumerate(medical_test):
            if x in query.lower().split(',') or x in query.lower().split(', '):
                count +=price_list[i]
                tests_append.append(x.title())
        return tests_append

In [16]:
def lab_date(query):
    global time_lab
    global date_lab
    doc = nlp(query.lower())
    for ent in doc.ents:
        if ent.label_=='DATE':
            date_lab = ent.text
        elif ent.label_=='TIME':
            time_lab = ent.text

In [17]:
def amount_to_pay(context):
    global pay
    if context == 'appointment':
        pay = 'Rs. 800'
    elif context == 'lab':
        pay = 'Rs. '+str(count)
    return pay

In [18]:
def get_name_age(query):
    global name
    global age
    doc = nlp(query)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            name = ent.text
            flag=True
        elif ent.label_ == 'DATE':
            age = ent.text
            return flag
    return False

In [19]:
def patient_details(query):
    global pat_name
    global pat_place
    global pat_age
    doc= nlp(query)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            pat_name = ent.text
        elif ((ent.label_ == 'LOC') or (ent.label_ == 'GPE')):
            pat_place = ent.text
        elif ent.label_ == 'DATE':
            pat_age = ent.text

In [20]:
def admission_date(query):
    global time_adm
    global date_adm
    global amount
    global ward_no
    doc = nlp(query.lower())
    card=0
    for ent in doc.ents:
        if ent.label_=='DATE':
            date_adm = ent.text.title()
        elif ent.label_=='TIME':
            time_adm = ent.text.title()
        elif ((ent.label_ == 'MONEY') or (ent.label_ == 'CARDINAL')) and card>0:
            amount = ent.text.title()
        elif ent.label_ == 'CARDINAL':
            ward_no = ent.text.title()
            card+=1

In [21]:
time=None
date=None
time_adm=None
date_adm=None
time_lab=None
date_lab=None
amount=None
place=None
name=None
age=None
pay=None
disease=None
days=None
stop = False
pat_name=None
pat_age=None
pat_place=None
amount=None
ward_no=None
limit=0
count=0
query = input("User > ")
while not stop:
    if(query.lower() == 'exit'):
        stop=True
    elif name:
        if context == 'appointment':
            if disease is not None:
                if days is not None:
                    if date is not None:
                        print("\nYour Appointment Details\nName: ", name, "\t Age: ", age, "\nDisease: ", disease, "\t Duration: ", days, 
                              "\nAppointment Date: ", date, "\t Time: ", time, "\nAmount to be paid: ", amount_to_pay(context), "\nSee You Soon!")
                        stop=True
                    else:
                        print("Bot > :) You have this symptom", disease, "from", days)
                        print("Bot > :) Please tell me, what date and time are you looking for an appointment?")
                        query = input("User > ")
                        appointment_date(query)
                else:
                    print("Bot > :) From how long you have this ", disease, "?")
                    query = input("User > ")
                    duration(query)
            else:
                if (limit == 0):
                    print("Bot > :) Could you tell me your symptoms please?")
                    query = input("User > ")
                    disease = detected_disease(query)
                    limit+=1
                elif(limit>=1 and limit <= 2):
                    print("Bot > :) Could you elaborate the symptom please?")
                    limit+=1
                    query = input("User > ")
                    disease = detected_disease(query)
                else:
                    print("Bot > :) Diagnosis for this symptom is not available in our hospital. Please look for another hospital. Thank you. Have a nice day!")
                    stop=True
        elif context == 'lab':
            if count!=0:
                if date_lab is not None:
                    print("\nYour Lab tests Details\nName: ", name, "\t Age: ", age, "\nTests: ", tests_append, "\nLab Tests Date: ", date_lab,
                              "\t Time: ", time_lab, "\nAmount to be Paid: ", amount_to_pay(context), "\nSee You Soon!")
                    stop=True
                else:
                    print("Bot > :) Could tell me what date and time are you looking for labtests?")
                    query = input("User > ")
                    lab_date(query)
            else:
                print("Bot > :) Are you looking for test or report?")
                query = input("User: ")
                if process.extract(query.lower(), ["test"])[0][1]>=90:
                    print("Bot > :) Here are the lab tests available in our Hospital\n", ", ".join(test for test in medical_test))
                    print("Bot > :) Could you fill the tests details as prescribed above.")
                    query = input("User > ")
                    tests_append = labtest_price(query)
                    if count!=0:
                        print("Bot > :) You have to pay: Rs.", count, "\n Is it ok for you?")
                        query =input("User > ") 
                        if (query.lower()!='yes'):
                            stop = True
                            print("Thank You. Have a Nice day!")
                else:
                    print("Please collect the report from the laboratory.\nThank You. Have a Nice Day!")
                    stop=True
        elif context == 'insurance':
            if pat_place is not None:
                if amount is not None:
                    print("Bot > :) The Details of the Patient for Insurance claim are")
                    print("\nPatient Name: ", pat_name, "\tAge: ", pat_age, "\tPlace: ", pat_place)
                    print("\nAdmission Date: ", date_adm, "\tTime:", time_adm, "\tWard No:", ward_no)
                    print("\nClaim Amount: ", amount)
                    print("\nThe details are forwarded to the Claim Department. Please contact the Claim Department. \nHave a nice Day!")
                    stop=True
                else:
                    print("Bot > :) Could you give Admission related details such as date, time, ward no and paid amount?")
                    query = input("User > ")
                    admission_date(query)
            else:
                if (limit == 0):
                    print("Bot > :) Could you give the details of the patient, such as name, age, and place to be claimed?")
                    query = input("User > ")
                    patient_details(query)
                    limit+=1
                else:
                    print("Bot > :) Please give the patient's details with the correct place name.")
                    query = input("User > ")
                    patient_details(query)
                    limit+=1
        else:
            print("Bot > :) Could you tell briefly what are you looking for?")
            query = input("User > ")
            max_score, best_match, context = get_item_match(query)   
    else:
        if(get_name_age(query)):
            print("Bot > :) Hi, ", name, "How can I help you?")
            query = input("User > ")
            max_score, best_match, context = get_item_match(query)
        else:
            print("Bot > :) Could you tell me your name and age please?")  
            query = input("User > ")

User > Hi
Bot > :) Could you tell me your name and age please?
User > I am Seetha. I am 30 years old.
Bot > :) Hi,  Seetha How can I help you?
User > I need to consult doctor
Bot > :) Could you tell me your symptoms please?
User > I have head ache
Bot > :) From how long you have this  Head Ache ?
User > from 3 days
Bot > :) You have this symptom Head Ache from 3 days
Bot > :) Please tell me, what date and time are you looking for an appointment?
User > i want an appointment on 02/04/2022 at 4:30 pm

Your Appointment Details
Name:  Seetha 	 Age:  30 years old 
Disease:  Head Ache 	 Duration:  3 days 
Appointment Date:  02/04/2022 	 Time:  4:30 pm 
Amount to be paid:  Rs. 800 
See You Soon!
